In [ ]:
%run conceptors.py

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:

# making some waveform patterns that we might use
pSaw = lambda n: (round(n % waveLengthSamples) / waveLengthSamples * 2) - 1.0
pPulse = lambda n: (((n % waveLengthSamples) < (waveLengthSamples * 0.5)) * 2) - 1.0
pSine2 = lambda n: (np.sin(n) * np.sin((n+np.pi/4)/6))
pSine3 = lambda n: (np.sin(n) * np.sin((n/4)/6)/6)
pJ1 = lambda n: 1 * np.sin(2 * np.pi * n / 3.1504531)
pJ1b = lambda n: 1 * np.sin(n/2) ** 1

period2 = 2
rawp = np.random.randn(period2)
# rawp = np.array([1.1929,2.6856]);
maxVal = np.max(rawp)
minVal = np.min(rawp)
rp = 0.5 * (2 * (rawp - minVal) / (maxVal - minVal) - 1);
pJ2 = lambda n: rp[np.mod(n, period2 )]
pTri = lambda n,p: (((n % p) >= (p/2)) * ((p/2) - (n % (p/2))) + ((n % p) < (p/2)) * (n % (p/2))) * (2/p)



In [ ]:
figsize(20,4)
# plot([pJ1b(x) for x in arange(15)])
# plot([pJ1(x) for x in arange(15)])
# plot([pJ2(x) for x in arange(15)])
plot([pSine2(x) for x in arange(15)])

# plot([pTri(x,10) for x in arange(10)])

In [ ]:
params = {'N':32,
          'Nout':2,
          'NetSR':0.6, # was 1.6; scaling for internal weights: according to ESN tutorial, this one should actually be < 1
          'NetinpScaling':1.6, # scaling for input weights
          'BiasScaling':0.3, # scaling for bias
          'TychonovAlpha':0.0001,
          'TychonovAlphaReadout':0.0001,
          'washoutLength':100, 'learnLength':500, 
          'learnLengthWout':500, 
          'recallTestLength':100,
          'alphas':np.array([12.0,24.0,36.0]),
          'patts':np.array([pJ1b, pJ2,pSine2])
}

net1 = makeNetwork(params)


In [ ]:
net2 = makeNetwork(params)
net3 = makeNetwork(params)


In [ ]:
morphValues = [1,0,0,1,0,0,1,0,0]

def onMorph(ind,f):
    global morphValues
#     print("onMorph", f)
    morphValues[ind] = f
    return morphValues

def onExit():
    print( "exiting" )
    global keepRunning
    keepRunning = False
    return True

oscserver = makeOSCServer(onMorph,onExit)

In [ ]:
import time

keepRunning = True

#run a network
x1 = 0.5 * np.random.randn(net1['p']['N'],1)
x2 = 0.5 * np.random.randn(net2['p']['N'],1)
x3 = 0.5 * np.random.randn(net3['p']['N'],1)

while keepRunning:
#     print( morphValues )
    # net 1
    output1, x1 = conceptor_mix_step( net1, x1, morphValues[0:3] )
    oscserver.send_value( "/output/1", output1[0][0] )
    oscserver.send_array( "/x/1", x1.flatten() )
    
    # net 2
    output2, x2 = conceptor_mix_step( net2, x2, morphValues[3:6] )
    oscserver.send_value( "/output/2", output2[0][0] )
    oscserver.send_array( "/x/2", x2.flatten() )

    # net 3
    output3, x3 = conceptor_mix_step( net3, x3, morphValues[6:9] )
    oscserver.send_value( "/output/3", output3[0][0] )
    oscserver.send_array( "/x/3", x3.flatten() )
    
    time.sleep(0.01)
    

In [ ]:
def conceptor_mix_step( net, x, morphvalues, oversample=1 ):
    ind = 0
    C = np.zeros( (net['p']['N'] , net['p']['N'] ) )
    for i_morph in morphvalues:
        C = C + (net['Cs'][0,ind].dot( i_morph ))
        ind = ind + 1
    Wsr = net['W'].dot(1.2)
    for i_oversample in range( oversample ):
        x = np.tanh(Wsr.dot(x) + net['Wbias'])
        x = C.dot(x)
    output = net1['Wout'].dot(x)
    return output, x


In [ ]:
print( net1['p']['N'] )
print( morphValues[0:3], morphValues[3:6], morphValues[6:9])
morphFactors = morphValues[0:3]
a, b = conceptor_mix_step( net1, x1, morphFactors )
print( a )
print( b )

In [ ]:
morphValues

In [ ]:
#test OSC
try:
    target = Address(57400)
except AddressError as err:
    print (str(err))
    sys.exit()

# send message "/foo/message1" with int, float and string arguments
send(target, "/morph", 0.3)

In [ ]:
a = [ 0, 2, 4]
oscserver.send_array("/morph", a)

In [ ]:
p = {'N':5,
          'Nout':2,
          'NetSR':0.6, # was 1.6; scaling for internal weights: according to ESN tutorial, this one should actually be < 1
          'NetinpScaling':1.6, # scaling for input weights
          'BiasScaling':0.3, # scaling for bias
          'TychonovAlpha':0.0001,
          'TychonovAlphaReadout':0.0001,
          'washoutLength':100, 'learnLength':500, 
          'learnLengthWout':500, 
          'recallTestLength':100,
          'alphas':np.array([12.0,24.0,36.0]),
          'patts':np.array([pJ1b, pJ2,pSine2])
}

In [ ]:
NetConnectivity = 1 # just for small networks
if p['N'] > 20:
    NetConnectivity = 10.0/p['N'];
WstarRaw = generateInternalWeights(p['N'], NetConnectivity)
WinRaw = np.random.randn(p['N'], 1) # input is 1-dimensional
WbiasRaw = np.random.randn(p['N'], 1) # bias is also 1-dimensional

#Scale raw weights
Wstar = p['NetSR'] * WstarRaw;
Win = p['NetinpScaling'] * WinRaw;
Wbias = p['BiasScaling'] * WbiasRaw;
I = np.eye(p['N']) # identity matrix

xCollector = np.zeros((p['N'],p['Nout'], p['learnLengthWout'])) # variable to collect states of x
pCollector = np.zeros((p['Nout'], p['learnLengthWout'])) # variable to collect states of p (output?)
x = np.zeros((p['N'],p['Nout'])) # initial state


# first training: washout is to wash out the input state 'noise'; learnLength is then the actual amount of learning samples
for n in np.arange(p['washoutLength'] + p['learnLength']):
    u = np.random.randn() * 1.5  # random input
    print( Win , u )
    print( Win.dot(u) )
    print( Wstar.dot(x) )
    x = np.tanh( Wstar.dot(x) + Win.dot(u) + Wbias ) # calculate next internal activation
    if n >= p['washoutLength']:
        xCollector[:,:, n - p['washoutLength']] = x[:,:]
        pCollector[:, n - p['washoutLength']] = u
        
print( pCollector.shape )

In [15]:
import os
cwd = os.getcwd()


gesture1 = np.loadtxt(cwd + '/workshop/gesturedata_0.txt')

print( gesture1 )

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [14]:
import os
cwd = os.getcwd()

cwd + '/workshop/gesturedata_0.txt'

/home/nescivi/git/projects/sussex/chrysalisConceptors


'/home/nescivi/git/projects/sussex/chrysalisConceptors/supercollider/workshop/gesturedata_0.txt'

In [ ]:

def makeNetwork(p):    
    NetConnectivity = 1 # just for small networks
    if p['N'] > 20:
        NetConnectivity = 10.0/p['N'];
    WstarRaw = generateInternalWeights(p['N'], NetConnectivity)
    WinRaw = np.random.randn(p['N'], p['Nout'])
    WbiasRaw = np.random.randn(p['N'], p['Nout'])

    #Scale raw weights
    Wstar = p['NetSR'] * WstarRaw;
    Win = p['NetinpScaling'] * WinRaw;
    Wbias = p['BiasScaling'] * WbiasRaw;
    I = np.eye(p['N']) # identity matrix
    
    xCollector = np.zeros((p['N'], p['learnLengthWout'])) # variable to collect states of x
    pCollector = np.zeros((p['Nout'], p['learnLengthWout'])) # variable to collect states of p (output?)
    x = np.zeros((p['N'],p['Nout'])) # initial state
    
    # first training: washout is to wash out the input state 'noise'; learnLength is then the actual amount of learning samples
    for n in np.arange(p['washoutLength'] + p['learnLength']):
        u = np.random.randn(p['Nout']) * 1.5  # random input
        x = np.tanh( Wstar.dot(x) + Win.dot(u) + Wbias ) # calculate next internal activation
        if n >= p['washoutLength']:
            xCollector[:, n - p['washoutLength']] = x[:,0]
            pCollector[:, n - p['washoutLength']] = u

#     print("Mean/Max/Min Activations, random network driven by noise")
#     plot(np.mean(xCollector.T, axis=1))
#     plot(np.max(xCollector.T, axis=1))
#     plot(np.min(xCollector.T, axis=1))

    # Wout 
    Wout = np.linalg.inv( xCollector.dot(xCollector.conj().T) + ( p['TychonovAlphaReadout'] * np.eye(p['N']) ) ).dot(xCollector).dot(pCollector.conj().transpose()).conj().T
    print("Initial training")
    print("NRMSE: ", nrmse(Wout.dot(xCollector), pCollector))
    print("absWeight: ", np.mean(abs(Wout)))

    allTrainArgs = np.zeros((p['N'], p['patts'].size * p['learnLength']))
    allTrainOldArgs = np.zeros((p['N'], p['patts'].size * p['learnLength']))
    allTrainTargs = np.zeros((p['N'], p['patts'].size * p['learnLength']))
    allTrainOuts = np.zeros((p['Nout'], p['patts'].size * p['learnLength']))
    xCollectors =  np.zeros((p['Nout'], p['patts'].size), dtype=np.object)
    SRCollectors =  np.zeros((p['Nout'], p['patts'].size), dtype=np.object)
    URCollectors =  np.zeros((p['Nout'], p['patts'].size), dtype=np.object)
    patternRs =  np.zeros((p['Nout'], p['patts'].size), dtype=np.object)
    #train_xPL =  np.zeros((1, p['patts'].size), dtype=np.object)
    #train_pPL =  np.zeros((1, p['patts'].size), dtype=np.object)
    startXs =  np.zeros((p['N'], p['patts'].size), dtype=np.object)

    for i_pattern in range(p['patts'].size):
        print('Loading pattern ', i_pattern)
        patt = p['patts'][i_pattern]
        xCollector = np.zeros((p['N'], p['learnLength']))
        xOldCollector = np.zeros((p['N'], p['learnLength']))
        pCollector = np.zeros((p['Nout'], p['learnLength']))
        x = np.zeros((p['N'],1))
        for n in range(p['washoutLength'] + p['learnLength']):
            u = patt(n+1)
            xOld = x
            x = np.tanh( Wstar.dot(x) + Win.dot(u) + Wbias )
            if n >= p['washoutLength']:
                xCollector[:, n - p['washoutLength']] = x[:,0]
                xOldCollector[:, n - p['washoutLength']] = xOld[:,0]
                pCollector[:, n - p['washoutLength']] = u

        xCollectors[0,i_pattern] = xCollector
        R = xCollector.dot(xCollector.T) / p['learnLength']
        [Ux,sx,Vx] = np.linalg.svd(R)
        SRCollectors[0,i_pattern] = np.diag(sx)
        URCollectors[0,i_pattern] = Ux
        patternRs[0,i_pattern] = R

        startXs[:,i_pattern] = x[:,0]

        #needed?
        #train_xPL[0,i_pattern] = xCollector[:,:signalPlotLength]
        #train_pPL[0,i_pattern] = pCollector[0,:signalPlotLength]
        ###

        allTrainArgs[:, i_pattern * p['learnLength']:(i_pattern+1) * p['learnLength']] = xCollector
        allTrainOldArgs[:, i_pattern * p['learnLength']:(i_pattern+1) * p['learnLength']] = xOldCollector
        allTrainOuts[:, i_pattern * p['learnLength']:(i_pattern+1) * p['learnLength']] = pCollector
        allTrainTargs[:, i_pattern * p['learnLength']:(i_pattern+1) * p['learnLength']] = Win.dot(pCollector)

    Wtargets = np.arctanh(allTrainArgs) - np.tile( Wbias, (1, p['patts'].size * p['learnLength']))

    W = np.linalg.inv(allTrainOldArgs.dot(allTrainOldArgs.conj().T) +
                      (p['TychonovAlpha'] * np.eye(p['N']))).dot(allTrainOldArgs).dot(Wtargets.conj().T).conj().T
    print("W NMRSE: ", np.mean(nrmse(W.dot(allTrainOldArgs), Wtargets)))
    print("absSize: ", np.mean(np.mean(abs(W), axis=0)))

    # figure(1)
    # plot(np.mean(W.dot(allTrainOldArgs).T, axis=1))

    print('Computing conceptors')

    Cs = np.zeros((4, p['patts'].size), dtype=np.object)
    for i_pattern in range(p['patts'].size):
        R = patternRs[0,i_pattern]
        [U,s,V] = np.linalg.svd(R)
        S = np.diag(s)
        Snew = (S * np.linalg.inv(S + pow(p['alphas'][i_pattern], -2) * np.eye(p['N'])))

        C =  U.dot(Snew).dot(U.T);
        Cs[0,i_pattern] = C
        Cs[1,i_pattern] = U
        Cs[2,i_pattern] = np.diag(Snew)
        Cs[3,i_pattern] = np.diag(S)

    x_CTestPL = np.zeros((3, p['recallTestLength'], p['patts'].size))
    p_CTestPL = np.zeros((1, p['recallTestLength'], p['patts'].size))
    for i_pattern in range(p['patts'].size):
        C = Cs[0,i_pattern]
        x = 0.5 * np.random.randn(p['N'],1)
        for n in range(p['recallTestLength'] + p['washoutLength']):
            x = np.tanh(W.dot(x) + Wbias)
            x = C.dot(x)
            if (n > p['washoutLength']):
                x_CTestPL[:,n-p['washoutLength'],i_pattern] = x[0:3].T
                p_CTestPL[:,n-p['washoutLength'],i_pattern] = Wout.dot(x)
    # for i_pattern in range(p['patts'].size):
    #     figure(2 + i_pattern)
    #     plot(p_CTestPL[:,:,i_pattern].T)
    #     plot([p['patts'][i_pattern](x) for x in arange(p['recallTestLength'])])

    return locals()